## Setup and Imports

In [ ]:
#!pip install gurobipy  # install gurobipy, if not already installed
#import gurobipy as gp  # import the installed package
#!pip install cplex
#!pip install docplex
!pip install pulp
import pulp as pl

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,LpMinimize
import pulp
from openpyxl import Workbook

## Create Classes

In [ ]:
class Blender:
    def __init__(self,location,type_blender,capacity):
        self.location=location
        self.type=type_blender
        self.capacity=capacity
    def  __repr__(self) :
        return "Blender in " + str(self.location)
class Recette:
  def __init__(self, nom, location, composition,quantite):
    self.nom = nom
    self.location = location
    self.composition = composition
    self.quantite = quantite
    def  __repr__(self) :
        return "Recette from " + str(self.location)
  
### Stockage class :
class Stockage:
    def __init__(self,location,capacity):
        self.location=location
        self.capacity=capacity
    def  __repr__(self) :
        return "Stockage in " + str(self.location)
### Supplier class :
class Supplier:
    def __init__(self,location,MP):
        self.location=location
        self.MP=MP
    def  __repr__(self) :
        return "Supplier in " + str(self.location)

## Read Data from the Excel File

In [ ]:
blender_file=pd.read_excel("small_data.xlsx",sheet_name="Blenders")
stockage_file=pd.read_excel("small_data.xlsx",sheet_name="Stockage")
supplier_file=pd.read_excel("small_data.xlsx",sheet_name="Capacité source",header=None)
path = 'small_data.xlsx'
file = pd.read_excel(path,sheet_name='Recettes')
file_demande = pd.read_excel(path,sheet_name='Demande')
mp=pd.read_excel("small_data.xlsx",sheet_name="MP")
cout_transport_amont=pd.read_excel("small_data.xlsx",sheet_name="Cout de transport  Amont")
cout_transport_aval=pd.read_excel("small_data.xlsx",sheet_name="Cout de transport aval")

## Creating Objects and Dataframes

From the data we have just imported we could new create objects and dataframes out of it for later access.

*   **Blenders**



In [ ]:
Capacity_new_smart_blender=3000
Capacity_new_blender=12000

l1=[]
l2=[]
l3=[]
for i in range(len(blender_file)):
    if blender_file.iloc[i, 1]==1:
        l1.append({"location":blender_file.iloc[i, 0],"type_blender":"B","capacity":blender_file.iloc[i, 3]})
    if blender_file.iloc[i, 2]==1:
        l1.append({"location":blender_file.iloc[i, 0],"type_blender":"SB","capacity":blender_file.iloc[i, 4]})
    if blender_file.iloc[i, 2]!=1  and blender_file.iloc[i, 1]!=1:
        l1.append({"location":blender_file.iloc[i, 0],"type_blender":"B","capacity":0})
blenders = pd.DataFrame(l1)

list_blenders=[]
for i in range(len(blenders)):
    list_blenders.append(Blender(blenders.iloc[i,0],blenders.iloc[i, 1],blenders.iloc[i, 2]))




*   **Recettes**






In [ ]:
n,m = file.shape
a,b = file_demande.shape
list_recettes = []
provinces = list(file['Province'])
col = file.columns 
col1 = file_demande.columns
periode = list(col1[2:6])
for i in range(n):
    obj = file.loc[i]
    dic1 ={}
    dict2={}
    for k in range(7):
        dic1[col[k+2]] = obj[k+2]
    for j in range(len(periode)): 
        store_df = file_demande
        store_df_filtered = store_df[store_df['Province'].isin([obj[0]])]
        quantite = float(store_df_filtered[store_df_filtered['Filière'].isin([obj[1]])][periode[j]])
        dict2[periode[j]] = quantite
    list_recettes.append(Recette(obj[1],obj[0],dic1,dict2))


*   **Stockages**


In [ ]:
list_stockages=[]
for i in range(len(stockage_file)):
    list_stockages.append(Stockage(stockage_file.iloc[i, 0],stockage_file.iloc[i, 1]))
list_stockages
for st in list_stockages :
  print(st.capacity)

600
0
500
0
22800


In [ ]:
list_recettes[0].quantite

{'Q1': 143.67180556145536, 'Q2': 23.945300926909223, 'Q3': 23.95, 'Q4': 287.34}

*   **Suppliers**

In [ ]:
list_suppliers=[]
for j in range(1,len(supplier_file.columns)):
    location=supplier_file.iloc[0, j]
    MP={}
    for i in range(1,len(supplier_file)):
        MP[supplier_file.iloc[i, 0]]=supplier_file.iloc[i, j]
    list_suppliers.append(Supplier(location,MP))
list_suppliers[0].MP

{'CAN': 0,
 'DAP': 508752000,
 'DURAMON 26%': 0,
 'EM': 508752000,
 'MAP': 508752000,
 'MOP': 0,
 'TSP': 0}

*   **Matieres premieres**

In [ ]:
prix_mp={}
for i in range(len(mp)):
  prix_mp[mp.iloc[i,0]]=mp.iloc[i,1]
mp = mp[mp.filter(regex='^(?!Unnamed)').columns]
for i in range(len(mp)):
    mp.rename(index={i: mp.iloc[i,0]}, inplace=True)
mp.drop('MP',axis='columns', inplace=True)
list_mp = list(mp.index)
list_mp

['CAN', 'DURAMON 26%', 'DAP', 'EM', 'MAP', 'MOP', 'TSP']

In [ ]:
prix_recette={}
for r in list_recettes :
  prix_recette[r]=sum(r.composition[i]*prix_mp[i] for i in prix_mp.keys())

*   **Transport_amont**

In [ ]:
cout_transport_amont = cout_transport_amont[cout_transport_amont.filter(regex='^(?!Unnamed)').columns]

for i in range(len(cout_transport_amont)):
    cout_transport_amont.rename(index={i: cout_transport_amont.iloc[i,0]}, inplace=True)
cout_transport_amont.drop('Zone',axis='columns', inplace=True)

cout_transport_amont

,OCP Jorf,OCP SAFI,OCP Port Casablanca,OCP Port Tanger,OCP Port Nador
Agadir Ida-Outanane,309.002807,279.524737,293.795789,411.051930,497.913333
Benslimane,127.905614,189.013509,95.957368,201.422281,288.274035
Casablanca,108.898867,170.006762,41.210482,193.118823,279.972990
Chefchaouen,286.265088,347.372982,254.312018,122.862018,343.648421
El Jadida,0.000000,138.513158,62.207895,179.464035,266.315789


*   **Transport_aval**

In [ ]:
cout_transport_aval= cout_transport_aval[cout_transport_aval.filter(regex='^(?!Unnamed)').columns]

for i in range(len(cout_transport_aval)):
    cout_transport_aval.rename(index={i: cout_transport_aval.iloc[i,0]}, inplace=True)
cout_transport_aval.drop('Zone',axis='columns', inplace=True)

cout_transport_aval

,Agadir Ida-Outanane,Benslimane,Casablanca,Chefchaouen,El Jadida
Agadir Ida-Outanane,27.789784,312.280000,284.280000,469.080000,252.920000
Benslimane,312.280000,28.434998,53.000000,194.680000,105.080000
Casablanca,284.280000,53.000000,22.958297,211.480000,82.680000
Chefchaouen,469.080000,194.680000,211.480000,29.294792,268.040000
El Jadida,252.920000,105.080000,82.680000,268.040000,29.191547


In [ ]:
periodes = [1, 2, 3, 4]

**Paramètres**

In [ ]:
Cout_de_stockage_MP =  10 #| Cost of stocking one unit of raw materials 
Cout_de_stockage_Engrais= 10 #| Cost of stocking one unit of final product
Capacite_stock_actif = 3500 #| Capacity of new storages
Capacite_blender_actiif = 12000 #| Capacity of new blenders
Capacite_smart_blender_actiif = 3000  #| Capacity of new smart blenders
Stockage_min = 100 #| Min limit of flow to open a new storage invest in a storage
Blender_min = 500   #| Min limit of flow to open a new storage invest in a blender
Cout_blender = 2600000 #| Investment cost for a single blender
Cout_sblender = 380000 #| Investment cost for a single smart blender
Cout_stockage_building = 10 #| Investment cost for building a single storage
Cout_blending = 120 
Cout_smart_blending = 120
Cout_de_rupture = 10**5

# Linear Optimisation

## Decision Variables

In [ ]:
provinces = []
for blender in list_blenders:
  if blender.location not in provinces:
    provinces.append(blender.location)
dic_blenders = {}
for province in provinces:
  L = []
  for blender in list_blenders:
    if blender.location == province :
      L.append(blender)
  dic_blenders[province] = L

In [ ]:

Supply_StorageMP = [(s,st,m,t)  for s in list_suppliers 
                              for st in list_stockages
                              for m in list_mp
                              for t in periodes]
Flow_Supply_StorageMP = LpVariable.dicts("Flow_Supply_StorageMP", Supply_StorageMP, lowBound=0)


StorageMP_Blender = [(st,b,m,t)  for st in list_stockages 
                              for b in list_blenders
                              for m in list_mp
                              for t in periodes]
Flow_StorageMP_Blender = LpVariable.dicts("Flow_StorageMP_Blender", StorageMP_Blender, lowBound=0)


Supply_Blender = [(s,b,m,t)  for s in list_suppliers 
                              for b in list_blenders
                              for m in list_mp
                              for t in periodes]
Flow_Supply_Blender = LpVariable.dicts("Flow_Supply_Blender", Supply_Blender, lowBound=0)

Blender_StorageFertlizer = [(b,st,r,t)  for b in list_blenders 
                              for st in list_stockages
                              for t in periodes
                              for r in list_recettes]
Flow_Blender_StorageFertlizer = LpVariable.dicts("Flow_Blender_StorageFertlizer", Blender_StorageFertlizer, lowBound=0)


StorageFertlizer_Province = [(st,r,t) for st in list_stockages
                              for t in periodes
                              for r in list_recettes]
Flow_StorageFertlizer_Province = LpVariable.dicts("Flow_StorageFertlizer_Province", StorageFertlizer_Province, lowBound=0)


Blender_Province = [(b,r,t) for b in list_blenders
                              for t in periodes
                              for r in list_recettes]
Flow_Blender_Province = LpVariable.dicts("Flow_StorageFertlizer_Province", Blender_Province, lowBound=0)



In [ ]:
stock_mp_list = [(st,m,t) for st in list_stockages
                           for m in list_mp
                           for t in periodes]
stock_mp_disponible = LpVariable.dicts("Stock_Mp_disponible", stock_mp_list, lowBound=0)

stock_pf_list = [(st,r,t) for st in list_stockages
                           for t in periodes
                           for r in list_recettes]
stock_pf_disponible = LpVariable.dicts("Stock_Pf_disponible", stock_pf_list, lowBound=0)


In [ ]:
recette_periode =[(r,t) for t in periodes
                  for r in list_recettes]
reliquat_demande = LpVariable.dicts("Reliquat demande", recette_periode)

In [ ]:
blender_periodes = [(b,t) for b in list_blenders
                    for t in periodes]
Active_Blender = LpVariable.dicts("Active_Blender" , blender_periodes, cat = "Binary")

Active_SBlender = LpVariable.dicts("Active_Smart_Blender" , blender_periodes, cat = "Binary")  

Blender_Cost = LpVariable.dicts("Investment in blender" , list_blenders, cat = "Binary")
SBlender_Cost = LpVariable.dicts("Investment in smart blender" , list_blenders, cat = "Binary")

In [ ]:
storage_periodes = [(st,t) for st in list_stockages
                    for t in periodes]
Storage_investement = LpVariable.dicts("Storage_investement" , storage_periodes, cat = "Binary")

Storage_Cost = LpVariable.dicts("Investment in storage" , list_stockages, cat = "Binary")

In [ ]:
Sum = LpVariable.dicts("Sum dakshi lli f la contrainte <=", storage_periodes)

## Create Model and Objective Function

In [ ]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMinimize)

#les couts à optimiser

#transport
cout_Flow_Supply_StorageMP = lpSum([cout_transport_amont.loc[st.location,s.location]*Flow_Supply_StorageMP[s,st,m,t] for (s,st,m,t) in Supply_StorageMP])
cout_Flow_Supply_Blender = lpSum([cout_transport_amont.loc[b.location , s.location]*Flow_Supply_Blender[s,b,m,t] for (s,b,m,t) in Supply_Blender])
cout_Flow_StorageMP_Blender = lpSum([cout_transport_aval.loc[st.location , b.location]*Flow_StorageMP_Blender[st,b,m,t] for (st,b,m,t) in StorageMP_Blender])
cout_Flow_Blender_StorageFertlizer = lpSum([cout_transport_aval.loc[b.location,st.location]*Flow_Blender_StorageFertlizer[b,st,r,t] for (b,st,r,t) in Blender_StorageFertlizer])
cout_Flow_StorageFertlizer_Province = lpSum([cout_transport_aval.loc[st.location,r.location]*Flow_StorageFertlizer_Province[st,r,t] for (st,r,t) in StorageFertlizer_Province])
cout_Flow_Blender_Province = lpSum([cout_transport_aval.loc[b.location,r.location]*Flow_Blender_Province[b,r,t] for (b,r,t) in Blender_Province])

#tranposrt cost
cout_transport = cout_Flow_Supply_StorageMP  + cout_Flow_Supply_Blender + cout_Flow_StorageMP_Blender + cout_Flow_Blender_StorageFertlizer + cout_Flow_StorageFertlizer_Province + cout_Flow_Blender_Province
#blending cost
cout_blending = lpSum([Cout_blending*Flow_Supply_Blender[s,b,m,t] if b.type == "B" else Cout_smart_blending*Flow_Supply_Blender[s,b,m,t] for (s,b,m,t) in Supply_Blender]) +\
                lpSum([Cout_blending*Flow_StorageMP_Blender[st,b,m,t] if b.type == "B" else Cout_smart_blending*Flow_StorageMP_Blender[st,b,m,t] for (st,b,m,t) in StorageMP_Blender])
#storage cost
cout_storage = lpSum([Cout_de_stockage_MP*stock_mp_disponible[st,m,t] for (st,m,t) in stock_mp_list ]) +\
                lpSum([Cout_de_stockage_Engrais*stock_pf_disponible[st,r, t] for (st,r,t) in stock_pf_list]) #+\
                #lpSum([Cout_de_stockage_MP*Flow_Supply_StorageMP[s,st,m,t]  for (s,st,m,t) in Supply_StorageMP]) +\
                #lpSum([Cout_de_stockage_Engrais * Flow_Blender_StorageFertlizer[b,st,r,t] for (b,st,r,t) in Blender_StorageFertlizer])
                
#cout rupture
cout_rupture_demande = lpSum(reliquat_demande[r,t] for (r,t) in recette_periode) 
#cout investissement
cout_investment_blender = lpSum([Cout_blender * Blender_Cost[b] for b in list_blenders]) + lpSum([Cout_sblender * SBlender_Cost[b] for b in list_blenders])
cout_investment_storage = lpSum([Cout_stockage_building * Storage_Cost[st] for st in list_stockages])
logistics =  cout_transport + cout_blending + cout_storage + cout_investment_blender + cout_investment_storage 
#fonction objective
objective = logistics + Cout_de_rupture* cout_rupture_demande 

model+= objective

## Constraints

In [ ]:
for b in list_blenders : 
  for t in periodes:
    model += Blender_Cost[b] >= Active_Blender[b,t]
    model += SBlender_Cost[b] >= Active_SBlender[b,t]
       
for b in list_blenders:
  model += Blender_Cost[b] <= 1
  model += SBlender_Cost[b] <= 1

In [ ]:
for st in list_stockages : 
  for t in periodes:
    model += Storage_Cost[st] >= Storage_investement[st,t]
       
for st in list_stockages :
  model += Storage_Cost[st] <= 1

In [ ]:
for t in periodes:
  for r in list_recettes : 
    constraint = r.quantite['Q' + str(t)] - lpSum([Flow_Blender_Province[b,r,t] for  b in list_blenders]) - \
    lpSum([Flow_StorageFertlizer_Province[st,r,t] for st in list_stockages]) == reliquat_demande[r,t]
    model += constraint

In [ ]:
for st in list_stockages:
  for m in list_mp:
    model += stock_mp_disponible[st,m,1] == lpSum(Flow_Supply_StorageMP[s,st,m,1] for s in list_suppliers ) - \
                                      lpSum(Flow_StorageMP_Blender[st,b,m,1] for b in list_blenders)
    model += lpSum(Flow_StorageMP_Blender[st,b,m,1] for b in list_blenders) == 0
    for t in periodes[1:]:
      model += stock_mp_disponible[st,m,t] == stock_mp_disponible[st,m,t-1] + lpSum(Flow_Supply_StorageMP[s,st,m,t] for s in list_suppliers ) - \
                                      lpSum(Flow_StorageMP_Blender[st,b,m,t] for b in list_blenders)
      model += lpSum(Flow_StorageMP_Blender[st,b,m,t] for b in list_blenders)<= stock_mp_disponible[st,m,t-1]

In [ ]:
for st in list_stockages:
    for r in list_recettes:
        model += stock_pf_disponible[st,r,1] == - Flow_StorageFertlizer_Province[st,r,1] + \
                                       lpSum(Flow_Blender_StorageFertlizer[b,st,r,1] for b in list_blenders)
        model += Flow_StorageFertlizer_Province[st,r,1] == 0
for t in periodes[1:]:  
  for st in list_stockages:
    for r in list_recettes:
      model += stock_pf_disponible[st,r,t] == stock_pf_disponible[st,r,t-1] - Flow_StorageFertlizer_Province[st,r,t] + \
                                       lpSum(Flow_Blender_StorageFertlizer[b,st,r,t] for b in list_blenders)
      model += Flow_StorageFertlizer_Province[st,r,t] <=  stock_pf_disponible[st,r,t-1]                            

In [ ]:
for t in periodes:
  for s in list_suppliers:
    for m in list_mp:
      model += (s.MP[m] >= lpSum(Flow_Supply_StorageMP[s,st,m,t] for st in list_stockages) + lpSum(Flow_Supply_Blender[s,b,m,t] for b in list_blenders)) 


In [ ]:
for t in periodes:
  for st in list_stockages:
    constraint =  lpSum(stock_mp_disponible[st,m,t] for m in list_mp) + \
    lpSum(stock_pf_disponible[st,r,t] for r in list_recettes) <= st.capacity + Capacite_stock_actif * Storage_investement[st,t]
    model += constraint

In [ ]:
for t in periodes:
  for r in list_recettes : 
    constraint = lpSum([Flow_Blender_Province[b,r,t] for  b in list_blenders]) + \
    lpSum([Flow_StorageFertlizer_Province[st,r,t] for st in list_stockages]) <= r.quantite['Q'+str(t)]
    model += constraint

In [ ]:
for t in periodes: 
  for b in list_blenders : 
    constraint = lpSum([Flow_Supply_Blender[s,b,m,t] for s in list_suppliers for  m in list_mp ]) +\
    lpSum([Flow_StorageMP_Blender[st,b,m,t] for st in list_stockages for m in list_mp ]) <=  +\
    b.capacity + Capacite_blender_actiif * Active_Blender[b,t] + Capacite_smart_blender_actiif *Active_SBlender[b,t]
    model += constraint

In [ ]:
for t in periodes: 
  for b in list_blenders : 
    constraint = lpSum([Flow_Supply_Blender[s,b,m,t] for s in list_suppliers for  m in list_mp ]) +\
    lpSum([Flow_StorageMP_Blender[st,b,m,t] for st in list_stockages for m in list_mp ]) >=  +\
    (b.capacity + Blender_min ) * (Active_Blender[b,t] + Active_SBlender[b,t])
    model += constraint

In [ ]:
for t in periodes: 
  for st in list_stockages : 
    constraint = lpSum(stock_mp_disponible[st,m,t] for m in list_mp) + \
    lpSum(stock_pf_disponible[st,r,t] for r in list_recettes) >=  +\
    (st.capacity + Stockage_min ) * Storage_investement[st,t]
    model += constraint

In [ ]:
for t in periodes:
  for b in list_blenders:  
    for m in list_mp:
      constraint = lpSum([Flow_Supply_Blender[s,b,m,t] for s in list_suppliers]) +\
      lpSum([Flow_StorageMP_Blender[st,b,m,t] for st in list_stockages  ]) - \
      lpSum([Flow_Blender_Province[b,r,t]*r.composition[m] for  r in list_recettes]) -\
      lpSum([Flow_Blender_StorageFertlizer[b,st,r,t]*r.composition[m] for st in list_stockages for r in list_recettes]) ==0
      model += constraint

In [ ]:
model.solve()

1

In [ ]:
cout_transport + cout_blending + cout_storage + cout_investment_blender + cout_investment_storage 

In [ ]:
logistics.value()

21936761.097967286

In [ ]:
#for t in periodes:
 # for r in list_recettes : 
 #   reliquat = reliquat_demande[r,t].value()
#    if reliquat <0 :
 #     print(reliquat)
     
for var in model.variables():
  if var.value() != 0:

    print(f"{var.name}: {var.value()}")


Active_Smart_Blender_(Blender_in_Agadir_Ida_Outanane,_1): 1.0
Active_Smart_Blender_(Blender_in_Agadir_Ida_Outanane,_4): 1.0
Active_Smart_Blender_(Blender_in_Chefchaouen,_1): 1.0
Active_Smart_Blender_(Blender_in_Chefchaouen,_2): 1.0
Active_Smart_Blender_(Blender_in_Chefchaouen,_3): 1.0
Active_Smart_Blender_(Blender_in_Chefchaouen,_4): 1.0
Active_Smart_Blender_(Blender_in_El_Jadida,_1): 1.0
Active_Smart_Blender_(Blender_in_El_Jadida,_2): 1.0
Active_Smart_Blender_(Blender_in_El_Jadida,_3): 1.0
Active_Smart_Blender_(Blender_in_El_Jadida,_4): 1.0
Flow_Blender_StorageFertlizer_(Blender_in_Chefchaouen,_Stockage_in_Chefchaouen,_<__main__.Recette_object_at_0x7f04458a56d0_,_3): 1520.19
Flow_Blender_StorageFertlizer_(Blender_in_Chefchaouen,_Stockage_in_Chefchaouen,_<__main__.Recette_object_at_0x7f0445b016d0_,_2): 159.21323
Flow_Blender_StorageFertlizer_(Blender_in_Chefchaouen,_Stockage_in_Chefchaouen,_<__main__.Recette_object_at_0x7f0445b016d0_,_3): 864.06677
Flow_Blender_StorageFertlizer_(Blende

In [ ]:
print(f"objective: {model.objective.value()}")

objective: 21936761.097967286


In [ ]:
print(logistics.value()) 

21936761.097967286


## Cost to market

In [ ]:
average_cost_storage_mp = {}
for periode in periodes:
  for storage in list_stockages:
    cost = lpSum([(storage == st and periode == t) *cout_transport_amont.loc[st.location,s.location]*Flow_Supply_StorageMP[s,st,m,t] for (s,st,m,t) in Supply_StorageMP]).value()
    volume = lpSum([(storage == st and periode == t) *Flow_Supply_StorageMP[s,st,m,t] for (s,st,m,t) in Supply_StorageMP]).value()
    average_cost_storage_mp[storage , periode] = cost/volume if volume > 0 else 0
average_cost_storage_mp

{(Stockage in Agadir Ida-Outanane, 1): 0,
 (Stockage in Agadir Ida-Outanane, 2): 0,
 (Stockage in Agadir Ida-Outanane, 3): 0,
 (Stockage in Agadir Ida-Outanane, 4): 0,
 (Stockage in Benslimane, 1): 0,
 (Stockage in Benslimane, 2): 0,
 (Stockage in Benslimane, 3): 0,
 (Stockage in Benslimane, 4): 0,
 (Stockage in Casablanca, 1): 0,
 (Stockage in Casablanca, 2): 0,
 (Stockage in Casablanca, 3): 0,
 (Stockage in Casablanca, 4): 0,
 (Stockage in Chefchaouen, 1): 0,
 (Stockage in Chefchaouen, 2): 0,
 (Stockage in Chefchaouen, 3): 0,
 (Stockage in Chefchaouen, 4): 0,
 (Stockage in El Jadida, 1): 0.0,
 (Stockage in El Jadida, 2): 0.0,
 (Stockage in El Jadida, 3): 0.0,
 (Stockage in El Jadida, 4): 0}

In [ ]:
average_cost_blender = {}
for periode in periodes :
  for blender in list_blenders: 
    cost = lpSum([(blender == b and periode == t) *cout_transport_amont.loc[b.location , s.location]*Flow_Supply_Blender[s,b,m,t] for (s,b,m,t) in Supply_Blender]).value() +\
            lpSum([(blender == b and periode == t)*(cout_transport_aval.loc[st.location , b.location] + average_cost_storage_mp[st,t] + Cout_de_stockage_MP)*Flow_StorageMP_Blender[st,b,m,t] for (st,b,m,t) in StorageMP_Blender]).value()
    volume = lpSum([(blender == b and periode == t) *Flow_Supply_Blender[s,b,m,t] for (s,b,m,t) in Supply_Blender]).value() +\
            lpSum([(blender == b and periode == t)*Flow_StorageMP_Blender[st,b,m,t] for (st,b,m,t) in StorageMP_Blender]).value()
    average_cost_blender[blender , periode] = cost /volume if volume > 0 else 0 
average_cost_blender    

{(Blender in Agadir Ida-Outanane, 1): 302.3972553516359,
 (Blender in Agadir Ida-Outanane, 2): 0,
 (Blender in Agadir Ida-Outanane, 3): 0,
 (Blender in Agadir Ida-Outanane, 4): 273.50430156977245,
 (Blender in Benslimane, 1): 0,
 (Blender in Benslimane, 2): 0,
 (Blender in Benslimane, 3): 0,
 (Blender in Benslimane, 4): 0,
 (Blender in Casablanca, 1): 73.44784486819563,
 (Blender in Casablanca, 2): 75.37372384235847,
 (Blender in Casablanca, 3): 75.3737000353923,
 (Blender in Casablanca, 4): 79.54419045401377,
 (Blender in Chefchaouen, 1): 207.50611964815926,
 (Blender in Chefchaouen, 2): 204.55068770916654,
 (Blender in Chefchaouen, 3): 200.12585122327621,
 (Blender in Chefchaouen, 4): 201.6524109751249,
 (Blender in El Jadida, 1): 48.8834932023284,
 (Blender in El Jadida, 2): 49.653138345977816,
 (Blender in El Jadida, 3): 44.734933063573784,
 (Blender in El Jadida, 4): 50.64973548870264}

In [ ]:
average_cost_storage_pf = {}
for periode in periodes: 
  for storage in list_stockages:
    cost = lpSum([(storage == st and periode == t)*(cout_transport_aval.loc[b.location,st.location]+ average_cost_blender[b,t] + (b.type =='SB')*Cout_smart_blending + (b.type =='B')*Cout_blending)*Flow_Blender_StorageFertlizer[b,st,r,t] for (b,st,r,t) in Blender_StorageFertlizer]).value()
    volume = lpSum([(storage == st and periode == t)*Flow_Blender_StorageFertlizer[b,st,r,t] for (b,st,r,t) in Blender_StorageFertlizer]).value()
    average_cost_storage_pf[storage , periode] = cost / volume if volume > 0 else 0
average_cost_storage_pf

{(Stockage in Agadir Ida-Outanane, 1): 0,
 (Stockage in Agadir Ida-Outanane, 2): 0,
 (Stockage in Agadir Ida-Outanane, 3): 417.6549330635738,
 (Stockage in Agadir Ida-Outanane, 4): 0,
 (Stockage in Benslimane, 1): 0,
 (Stockage in Benslimane, 2): 0,
 (Stockage in Benslimane, 3): 0,
 (Stockage in Benslimane, 4): 0,
 (Stockage in Casablanca, 1): 0,
 (Stockage in Casablanca, 2): 0,
 (Stockage in Casablanca, 3): 0,
 (Stockage in Casablanca, 4): 0,
 (Stockage in Chefchaouen, 1): 0,
 (Stockage in Chefchaouen, 2): 353.84547924638764,
 (Stockage in Chefchaouen, 3): 349.42064276049734,
 (Stockage in Chefchaouen, 4): 0,
 (Stockage in El Jadida, 1): 0,
 (Stockage in El Jadida, 2): 198.84468567062555,
 (Stockage in El Jadida, 3): 193.92648038822153,
 (Stockage in El Jadida, 4): 0}

In [ ]:
cost_to_market = {}
for periode in periodes :
  for recette in list_recettes:
    cost = lpSum([(recette == r and periode == t)*(cout_transport_aval.loc[b.location,r.location] + average_cost_blender[b,t] + (b.type =='SB')*Cout_smart_blending + (b.type =='B')*Cout_blending)*Flow_Blender_Province[b,r,t] for (b,r,t) in Blender_Province]).value() +\
          lpSum([(recette == r and periode == t)*(cout_transport_aval.loc[st.location,r.location] + average_cost_storage_pf[st,t] + Cout_de_stockage_Engrais)*Flow_StorageFertlizer_Province[st,r,t] for (st,r,t) in StorageFertlizer_Province]).value() +\
          sum([recette.quantite["Q" + str(periode)] * recette.composition[m]*mp.loc[m,'Prix'] for m in list_mp])
    volume = lpSum([(recette == r and periode == t)*Flow_Blender_Province[b,r,t] for (b,r,t) in Blender_Province]).value() +\
          lpSum([(recette == r and periode == t)*Flow_StorageFertlizer_Province[st,r,t] for (st,r,t) in StorageFertlizer_Province]).value()
    cost = cost/volume  if volume >0 else 0
    cost_to_market[recette, periode] = cost * (0.06 + 0.08 + 1)
cost_to_market
  

{(<__main__.Recette at 0x7f04456dd590>, 1): 2742.522566872259,
 (<__main__.Recette at 0x7f04456dd590>, 2): 2733.486106359611,
 (<__main__.Recette at 0x7f04456dd590>, 3): 2727.8793505401313,
 (<__main__.Recette at 0x7f04456dd590>, 4): 2634.5778063512894,
 (<__main__.Recette at 0x7f04456dd710>, 1): 2904.846486270657,
 (<__main__.Recette at 0x7f04456dd710>, 2): 2873.3667058229216,
 (<__main__.Recette at 0x7f04456dd710>, 3): 2867.759959101217,
 (<__main__.Recette at 0x7f04456dd710>, 4): 2871.9085928455834,
 (<__main__.Recette at 0x7f04456dd790>, 1): 2864.6608467958004,
 (<__main__.Recette at 0x7f04456dd790>, 2): 2833.180963474289,
 (<__main__.Recette at 0x7f04456dd790>, 3): 2827.5742611762735,
 (<__main__.Recette at 0x7f04456dd790>, 4): 2752.852394530211,
 (<__main__.Recette at 0x7f04456dd7d0>, 1): 2940.547887993866,
 (<__main__.Recette at 0x7f04456dd7d0>, 2): 2852.4451788439733,
 (<__main__.Recette at 0x7f04456dd7d0>, 3): 2846.8383453757756,
 (<__main__.Recette at 0x7f04456dd7d0>, 4): 285

In [ ]:
cost_market_province = {}
provinces = cout_transport_aval.columns
nombre_recette_province= {}
for periode in periodes : 
  for province in  provinces : 
    cost_market_province[province, periode] =0
    nombre_recette_province[province,periode] = 0
for periode in periodes :
  for recette in list_recettes:
    cost_market_province[recette.location ,periode] += cost_to_market[r,periode]
    #print(recette.location)
    nombre_recette_province[recette.location, periode] += 1
    #print(recette.location , cost_market_province[recette.location] )
for periode in periodes:
  for province in  provinces : 
    cost_market_province[province,periode] = cost_market_province[province,periode] / nombre_recette_province[recette.location, periode] 
cost_market_province

{('Agadir Ida-Outanane', 1): 2012.064577588459,
 ('Agadir Ida-Outanane', 2): 1950.3926411328757,
 ('Agadir Ida-Outanane', 3): 1946.467882299298,
 ('Agadir Ida-Outanane', 4): 1951.187894634551,
 ('Benslimane', 1): 2299.5023743868105,
 ('Benslimane', 2): 2229.0201612947153,
 ('Benslimane', 3): 2224.534722627769,
 ('Benslimane', 4): 2229.929022439487,
 ('Casablanca', 1): 1724.6267807901077,
 ('Casablanca', 2): 1671.7651209710361,
 ('Casablanca', 3): 1668.401041970827,
 ('Casablanca', 4): 1672.4467668296152,
 ('Chefchaouen', 1): 2586.9401711851615,
 ('Chefchaouen', 2): 2507.647681456555,
 ('Chefchaouen', 3): 2502.6015629562403,
 ('Chefchaouen', 4): 2508.670150244423,
 ('El Jadida', 1): 2874.377967983513,
 ('El Jadida', 2): 2786.275201618394,
 ('El Jadida', 3): 2780.6684032847115,
 ('El Jadida', 4): 2787.4112780493588}

Model Output

In [ ]:
result = Workbook()

In [ ]:


"""## Cost to supply"""

sheet = result.active
sheet.title = "Cost_to_supply"
header = ['Supply', 'MP', 'periode', 'volume_s' , 'cost_s', 'average_cost_supply']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
Supply = [(s,m,t) for s in list_suppliers for m in list_mp for t in periodes]
i=2
for supplier in Supply :
    c1=sheet.cell(row=i, column=1, value= supplier[0].location)
    c2=sheet.cell(row=i, column=2, value= supplier[1])
    c3=sheet.cell(row=i, column=3, value= 'Q'+str(supplier[2]))
    volume_s=sum(Flow_Supply_StorageMP[supplier[0],st,supplier[1],supplier[2]].value() for st in list_stockages) + sum(Flow_Supply_Blender[supplier[0],b,supplier[1],supplier[2]].value() for b in list_blenders)
    c4=sheet.cell(row=i, column=4, value= volume_s)
    cost_s=sum(prix_mp[supplier[1]]*Flow_Supply_StorageMP[supplier[0],st,supplier[1],supplier[2]].value() for st in list_stockages) 
    + sum(prix_mp[supplier[1]]*Flow_Supply_Blender[supplier[0],b,supplier[1],supplier[2]].value() for b in list_blenders)
    c5=sheet.cell(row=i, column=5, value= cost_s)
    if(volume_s!=0) :
        c6=sheet.cell(row=i, column=6, value= cost_s/volume_s)
    i+=1

"""## Flow from supply to storage MP"""

sheet = result.create_sheet()
sheet.title = "flow_supply_stockage_mp"
header = ['Supply', 'Stockage_MP', 'Matiere_premiere', 'Periode', 'Quantité', 'freight_cost', 'cost_stock_total', 'average_cost_stock_mp']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for element in Supply_StorageMP :
    c1=sheet.cell(row=i, column=1, value=element[0].location)
    c2=sheet.cell(row=i, column=2, value=element[1].location)
    c3=sheet.cell(row=i, column=3, value=element[2])
    c4=sheet.cell(row=i, column=4, value='Q'+str(element[3]))
    Quantité=Flow_Supply_StorageMP[element].value()
    c5=sheet.cell(row=i, column=5, value=Quantité)
    freight_cost=cout_transport_amont.loc[element[1].location,element[0].location]
    c6=sheet.cell(row=i, column=6, value=freight_cost)
    cost_stock_total=Quantité*(prix_mp[element[2]]+freight_cost)
    c7=sheet.cell(row=i, column=7, value=cost_stock_total)
    if(Quantité!=0) :
        c8=sheet.cell(row=i, column=8, value=cost_stock_total/Quantité)
    i+=1

"""## Flow from supply to blenders"""

sheet = result.create_sheet()
sheet.title = "flow_supply_blender"
header = ['Supply', 'Blender', 'Matiere_premiere', 'Periode', 'Type of Blender', 'Quantité', 'freight_cost', 'cost_blender_total', 'average_cost_blender']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for element in Supply_Blender :
    c1=sheet.cell(row=i, column=1, value=element[0].location)
    c2=sheet.cell(row=i, column=2, value=element[1].location)
    c3=sheet.cell(row=i, column=3, value=element[2])
    c4=sheet.cell(row=i, column=4, value='Q'+str(element[3]))
    c5=sheet.cell(row=i, column=5, value=element[1].type)
    Quantité=Flow_Supply_Blender[element].value()
    c6=sheet.cell(row=i, column=6, value=Quantité)
    freight_cost=cout_transport_amont.loc[element[1].location,element[0].location]
    c7=sheet.cell(row=i, column=7, value=freight_cost)
    cost_blender_total=Quantité*(prix_mp[element[2]]+freight_cost)
    c8=sheet.cell(row=i, column=8, value=cost_blender_total)
    if(Quantité!=0) :
        c9=sheet.cell(row=i, column=9, value=cost_blender_total/Quantité)
    i+=1

"""## Flow storage MP to blender"""

sheet = result.create_sheet()
sheet.title = "flow_stockage_mp_blender"
header = ['Stockage_MP', 'Blender', 'Matiere_premiere', 'Periode', 'Type of Blender', 'Quantité', 'freight_cost', 'cost_stock_blender_total', 'average_cost_stock_blender']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for element in StorageMP_Blender :
    c1=sheet.cell(row=i, column=1, value=element[0].location)
    c2=sheet.cell(row=i, column=2, value=element[1].location)
    c3=sheet.cell(row=i, column=3, value=element[2])
    c4=sheet.cell(row=i, column=4, value='Q'+str(element[3]))
    c5=sheet.cell(row=i, column=5, value=element[1].type)
    Quantité=Flow_StorageMP_Blender[element].value()
    c6=sheet.cell(row=i, column=6, value=Quantité)
    freight_cost=cout_transport_aval.loc[element[0].location,element[1].location]
    c7=sheet.cell(row=i, column=7, value=freight_cost)
    cost_stock_blender_total=Quantité*(prix_mp[element[2]]+freight_cost)
    c8=sheet.cell(row=i, column=8, value=cost_stock_blender_total)
    if(Quantité!=0) :
        c9=sheet.cell(row=i, column=9, value=cost_stock_blender_total/Quantité)
    i+=1

"""## Cost into blenders"""

sheet = result.create_sheet()
sheet.title = "cost_to_blender"
header = ['Blender', 'MP', 'periode', 'Type of Blender', 'volume_ocp', 'cost_ocp', 'volume_st', 'cost_st', 'average_cost_b']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
Blenders = [(b,m,t) for b in list_blenders for m in list_mp for t in periodes]
i=2
for blender in Blenders :
    c1=sheet.cell(row=i, column=1, value=blender[0].location)
    c2=sheet.cell(row=i, column=2, value=blender[1])
    c3=sheet.cell(row=i, column=3, value='Q'+str(blender[2]))
    c4=sheet.cell(row=i, column=4, value=blender[0].type)
    volume_ocp=sum(Flow_Supply_Blender[s,blender[0],blender[1],blender[2]].value() for s in list_suppliers)
    c5=sheet.cell(row=i, column=5, value=volume_ocp)
    cost=Cout_blending*Active_Blender[blender[0],blender[2]].value()+Cout_smart_blending*Active_SBlender[blender[0],blender[2]].value()
    cost_ocp=sum((cout_transport_amont.loc[blender[0].location,s.location]+cost+prix_mp[blender[1]])*Flow_Supply_Blender[s,blender[0],blender[1],blender[2]].value() for s in list_suppliers)
    c6=sheet.cell(row=i, column=6, value=cost_ocp)
    volume_st=sum(Flow_StorageMP_Blender[st,blender[0],blender[1],blender[2]].value() for st in list_stockages)
    c7=sheet.cell(row=i, column=7, value=volume_st)
    cost_st=sum((cout_transport_aval.loc[st.location,blender[0].location]+cost+prix_mp[blender[1]]+Cout_de_stockage_MP)*Flow_StorageMP_Blender[st,blender[0],blender[1],blender[2]].value() for st in list_stockages)
    c8=sheet.cell(row=i, column=8, value=cost_st)
    if(volume_ocp+volume_st!=0) :
        c9=sheet.cell(row=i, column=9, value=(cost_ocp+cost_st)/(volume_ocp+volume_st))
    i+=1

"""## Flow from blender to storage"""

sheet = result.create_sheet()
sheet.title = "flow_blender_stockage"
header = ['Blender', 'Stockage_Engrais', 'Province', 'Culture', 'Periode', 'Quantité', 'freight_cost', 'cost_to_stock', 'average_cost_st']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for element in Blender_StorageFertlizer :
    c1=sheet.cell(row=i, column=1, value=element[0].location)
    c2=sheet.cell(row=i, column=2, value=element[1].location)
    c3=sheet.cell(row=i, column=3, value=element[2].location)
    c4=sheet.cell(row=i, column=4, value=element[2].nom)
    c5=sheet.cell(row=i, column=5, value='Q'+str(element[3]))
    Quantité=Flow_Blender_StorageFertlizer[element].value()
    c6=sheet.cell(row=i, column=6, value=Quantité)
    freight_cost=cout_transport_aval.loc[element[0].location,element[1].location]
    c7=sheet.cell(row=i, column=7, value=freight_cost)
    cost_to_stock=Quantité*(prix_recette[element[2]]+freight_cost)
    c8=sheet.cell(row=i, column=8, value=cost_to_stock)
    if(Quantité!=0) :
        c9=sheet.cell(row=i, column=9, value=cost_to_stock/Quantité)
    i+=1

"""## Flow from blender to province"""

sheet = result.create_sheet()
sheet.title = "flow_blender_province"
header = ['Blender', 'Province', 'Culture', 'Periode', 'Type of Blender', 'Quantité', 'freight_cost', 'cost_to_province', 'average_cost_p']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for element in Blender_Province :
    c1=sheet.cell(row=i, column=1, value=element[0].location)
    c2=sheet.cell(row=i, column=2, value=element[1].location)
    c3=sheet.cell(row=i, column=3, value=element[1].nom)
    c4=sheet.cell(row=i, column=4, value='Q'+str(element[2]))
    c5=sheet.cell(row=i, column=5, value=element[0].type)
    Quantité=Flow_Blender_Province[element].value()
    c6=sheet.cell(row=i, column=6, value=Quantité)
    freight_cost=cout_transport_aval.loc[element[0].location,element[1].location]
    c7=sheet.cell(row=i, column=7, value=freight_cost)
    cost_to_province=Quantité*(freight_cost+prix_recette[element[1]])
    c8=sheet.cell(row=i, column=8, value=cost_to_province)
    if(Quantité!=0) :
        c9=sheet.cell(row=i, column=9, value=cost_to_province/Quantité)
    i+=1

"""## Flow from storage to province"""

sheet = result.create_sheet()
sheet.title = "flow_stockage_province"
header = ['Stockage_Engrais', 'Province', 'Culture', 'Periode', 'Quantité', 'freight_cost', 'average_cost_p', 'Total_cost']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for element in StorageFertlizer_Province :
    c1=sheet.cell(row=i, column=1, value=element[0].location)
    c2=sheet.cell(row=i, column=2, value=element[1].location)
    c3=sheet.cell(row=i, column=3, value=element[1].nom)
    c4=sheet.cell(row=i, column=4, value='Q'+str(element[2]))
    Quantité=Flow_StorageFertlizer_Province[element].value()
    c5=sheet.cell(row=i, column=5, value=Quantité)
    freight_cost=cout_transport_aval.loc[element[0].location,element[1].location]
    c6=sheet.cell(row=i, column=6, value=freight_cost)
    Total_cost=Quantité*(freight_cost+prix_recette[element[1]])
    c8=sheet.cell(row=i, column=8, value=Total_cost)
    if(Quantité!=0) :
        c7=sheet.cell(row=i, column=7, value=Total_cost/Quantité)
    i+=1

"""## Cost to province"""

sheet = result.create_sheet()
sheet.title = "Cost_to_province"
header = ['province', 'culture', 'periode', 'volume_b', 'cost_b', 'volume_se', 'cost_se', 'average_cost_p', 'volume_total', 'marge_distr', 'marge_revend']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for recette in recette_periode :
    c1=sheet.cell(row=i, column=1, value=recette[0].location)
    c2=sheet.cell(row=i, column=2, value=recette[0].nom)
    c3=sheet.cell(row=i, column=3, value='Q'+str(recette[1]))
    volume_b=sum(Flow_Blender_Province[b,recette[0],recette[1]].value() for b in list_blenders)
    c4=sheet.cell(row=i, column=4, value=volume_b)
    cost_b=sum((cout_transport_aval.loc[b.location,recette[0].location]+prix_recette[recette[0]])*Flow_Blender_Province[b,recette[0],recette[1]].value() for b in list_blenders)
    c5=sheet.cell(row=i, column=5, value=cost_b)
    volume_se=sum(Flow_StorageFertlizer_Province[st,recette[0],recette[1]].value() for st in list_stockages)
    c6=sheet.cell(row=i, column=6, value=volume_se)
    cost_se=sum((cout_transport_aval.loc[st.location,recette[0].location]+prix_recette[recette[0]]+Cout_de_stockage_Engrais)*Flow_StorageFertlizer_Province[st,recette[0],recette[1]].value() for st in list_stockages)
    c7=sheet.cell(row=i, column=7, value=cost_se)
    c9=sheet.cell(row=i, column=9, value=volume_b+volume_se)
    if(volume_b+volume_se!=0):
        average_cost_p=(cost_b+cost_se)/(volume_b+volume_se)
        c8=sheet.cell(row=i, column=8, value=average_cost_p)
        c10=sheet.cell(row=i, column=10, value=average_cost_p*1.06)
        c11=sheet.cell(row=i, column=11, value=average_cost_p*1.08)
    i+=1

"""## Stock of raw materials"""

sheet = result.create_sheet()
sheet.title = "stock_fin_periode_mp"
header = ['Stockage_MP', 'Matiere premiere', 'Periode', 'Quantité']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for stock in stock_mp_list :
    c1=sheet.cell(row=i, column=1, value=stock[0].location)
    c2=sheet.cell(row=i, column=2, value=stock[1])
    c3=sheet.cell(row=i, column=3, value='Q'+str(stock[2]))
    Quantité=stock_mp_disponible[stock].value()
    c4=sheet.cell(row=i, column=4, value=Quantité)
    i+=1

"""## Stock of final products"""

sheet = result.create_sheet()
sheet.title = "stock_fin_periode_engrais"
header = ['Stockage_Engrais', 'Province', 'Culture', 'Periode', 'Quantité', 'cost']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for stock in stock_pf_list :
    c1=sheet.cell(row=i, column=1, value=stock[0].location)
    c2=sheet.cell(row=i, column=2, value=stock[1].location)
    c3=sheet.cell(row=i, column=3, value=stock[1].nom)
    c4=sheet.cell(row=i, column=4, value='Q'+str(stock[2]))
    Quantité=stock_pf_disponible[stock].value()
    c5=sheet.cell(row=i, column=5, value=Quantité)
    cost=Cout_de_stockage_Engrais*Quantité
    c6=sheet.cell(row=i, column=6, value=cost)
    i+=1

"""## Reliquat_Demand"""

sheet = result.create_sheet()
sheet.title = "reliquat"
header = ['Province', 'Culture', 'Periode', 'Quantité', 'cost']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for reliquat in recette_periode :
    c1=sheet.cell(row=i, column=1, value=reliquat[0].location)
    c2=sheet.cell(row=i, column=2, value=reliquat[0].nom)
    c3=sheet.cell(row=i, column=3, value='Q'+str(reliquat[1]))
    Quantité=reliquat_demande[reliquat].value()
    c4=sheet.cell(row=i, column=4, value=Quantité)
    cost=Cout_de_rupture*Quantité
    c5=sheet.cell(row=i, column=5, value=cost)
    i+=1

"""## Activation of blenders and smart blenders

### Active_B
"""

sheet = result.create_sheet()
sheet.title = "Active_B"
header = ['Blender', 'periode', 'Active_b']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for blender in blender_periodes :
    c1=sheet.cell(row=i, column=1, value=blender[0].location)
    c2=sheet.cell(row=i, column=2, value='Q'+str(blender[1]))
    c3=sheet.cell(row=i, column=3, value=Active_Blender[blender].value())
    i+=1

"""### Active_SB"""

sheet = result.create_sheet()
sheet.title = "Active_SB"
header = ['Blender', 'periode', 'Active_sb']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for blender in blender_periodes :
    c1=sheet.cell(row=i, column=1, value=blender[0].location)
    c2=sheet.cell(row=i, column=2, value='Q'+str(blender[1]))
    c3=sheet.cell(row=i, column=3, value=Active_SBlender[blender].value())
    i+=1

"""## Activation of upstream and downstream warehouses

### Active_stock_amont
"""

sheet = result.create_sheet()
sheet.title = "Active_stock_amont"
header = ['Stockage_MP', 'periode', 'Active_St']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for stock in storage_periodes :
    c1=sheet.cell(row=i, column=1, value=stock[0].location)
    c2=sheet.cell(row=i, column=2, value='Q'+str(stock[1]))
    c3=sheet.cell(row=i, column=3, value=Storage_investement[stock].value())
    i+=1

"""### Active_stock_aval"""

sheet = result.create_sheet()
sheet.title = "Active_stock_aval"
header = ['Stockage_aval', 'periode', 'Active_St']
i=1
for head in header :
    cell = sheet.cell(row=1, column=i, value=head)
    i+=1
i=2
for stock in storage_periodes :
    c1=sheet.cell(row=i, column=1, value=stock[0].location)
    c2=sheet.cell(row=i, column=2, value='Q'+str(stock[1]))
    c3=sheet.cell(row=i, column=3, value=Storage_investement[stock].value())
    i+=1

"""## Saving the Excel file"""

result.save("Result.xlsx")